In [1]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup
import re
import random


In [2]:
conn = sqlite3.connect('cracked.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS archive_urls
            (original TEXT, mimetype TEXT, timestamp TEXT, endtimestamp TEXT, groupcount integer, uniqcount integer)''')
cursor.execute('''CREATE TABLE IF NOT EXISTS topics
            (forum text,url TEXT, title TEXT, author TEXT)''')
cursor.execute('''CREATE TABLE IF NOT EXISTS posts
            (url text,title TEXT,user_id, author TEXT, body TEXT,timestamp TEXT)''')
conn.commit()

In [3]:
""" import json

# Load the JSON data
with open(r'C:\Users\Aston\Documents\GitHub\web_scraper\cracked.io\archived.json', 'r') as file:
    json_data = json.load(file)

# Insert the data into the SQLite table
for row in json_data:
    cursor.execute('INSERT INTO archive_urls VALUES (?, ?, ?, ?, ?, ?)', row)

conn.commit() """

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Aston\\Documents\\GitHub\\web_scraper\\cracked.io\\archived.json'

In [4]:
user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
]

In [5]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS urls (url TEXT)''')
    c.commit()
create_tables(conn) 

In [6]:
def scrape_forums(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    all_links = soup.find_all("a")
    pattern = re.compile(r".*Forum-.*")

    # Filter and print the links that match the pattern
    for link in all_links:
        href = link.get("href")
        if href and pattern.match(href):
            cursor.execute('''INSERT INTO urls (url) VALUES (?)''', (href,))
            conn.commit()


In [8]:
scrape_forums("https://cracked.io/")
conn.close()

In [9]:
def scrape_topics(url):
    print(f"scraping: {url[0]}")
    archive_url = f"https://cracked.io/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    topics = soup.find_all("tr", class_="inline_row")
    for topic in topics:
        try:
            topic_url = topic.find("span", class_="subject_new").find("a").get("href")
            print(f"url: {topic_url}")
            title = topic.find("span", class_="subject_new").text.strip()
            print(f"Title: {title}")
            author = topic.find("div", class_="author").text.strip().split(" ")[0]
            print(f"author: {author}")
            # Apply to sqlite
            cursor.execute("INSERT or ignore INTO topics VALUES (?,?,?,?)",(url[0],topic_url, title, author))
            conn.commit()
        except:
            continue


In [10]:
conn = sqlite3.connect('cracked.db')
cursor = conn.cursor()

cursor.execute('''select url from urls''')
urls = cursor.fetchall()

for url in urls:
    scrape_topics(url)

scraping: Forum-Announcements
scraping: Forum-MOTQ
scraping: Forum-Changelogs
scraping: Forum-Feedback-Suggestions
scraping: Forum-Accepted-Suggestions
scraping: Forum-Denied-Suggestions
scraping: Forum-Upgraded-Tools
scraping: Forum-Exclusive-Releases
scraping: Forum-Outdated-Releases
scraping: Forum-Request
scraping: Forum-Non-Auth
scraping: Forum-Forum-Support-Bugs
scraping: Forum-Staff-support
scraping: Forum-Solved--131
scraping: Forum-General-Hacking
scraping: Forum-Hacking-Tools-and-Programs
scraping: Forum-Hacking-Tutorials
scraping: Forum-Website-Hacking
scraping: Forum-SQL-Injection-Attacks
scraping: Forum-Requests-for-Hacking
scraping: Forum-Lounge
scraping: Forum-Introduction
scraping: Forum-LQ-Lounge
scraping: Forum-Gaming
scraping: Forum-First-Person-Shooters
scraping: Forum-Fortnite
scraping: Forum-RPG-MMPORG-games
scraping: Forum-League-of-Legends
scraping: Forum-Strategy-games
scraping: Forum-Game-discussions
scraping: Forum-Personal
scraping: Forum-Finance
scraping: F

In [11]:
def scrape_posts(url):
    print(f"scraping: {url[0]}")
    archive_url = f"https://cracked.io/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    posts = soup.find_all("div", class_="post-box")
    for post in posts:
        try:
            #print(post)
            user_id = post.find("div", class_="post-username").find("a").get("href").split("/")[-1]
            #print(f"user_id: {user_id}")
            author = post.find("div", class_="post-username").text.strip()
            #print(f"author: {author}")
            body = post.find("div", class_="post_body").text.strip()
            #print(f"body: {body}")
            timestamp = post.find("span", class_="post_date").text.strip()
            #print(f"timestamp: {timestamp}")
            # Apply to sqlite
            cursor.execute("INSERT or ignore INTO posts VALUES (?,?,?,?,?,?)",(url[0],url[1],user_id,author, body,timestamp))
            conn.commit()
            print(f"Inserted: {url[1]} {user_id}")
        except Exception as e:
            print(e)
            continue

In [12]:
conn = sqlite3.connect('cracked.db')
cursor = conn.cursor()

cursor.execute('''select topics.url, topics.title from topics left join posts on posts.url = topics.url where posts.url is NULL''')
topics = cursor.fetchall()

for topic in topics:
    scrape_posts(topic)
conn.close()

scraping: Thread-FREE-ROBLOX-COOKIE
scraping: Thread-HOW-TO-CRACK-ACCOUNTS-WITH-%E2%9C%94%EF%B8%8F%E2%9C%94%EF%B8%8FOPENBULLET%E2%9C%94%EF%B8%8F%E2%9C%94%EF%B8%8F-FOR-BEGINNERS
scraping: Thread-%E2%9C%85%EF%B8%8F-FREE-MASTER-DOXXING-AND-OSINT-TOOL-WITH-GUI-ANYONE-CAN-DOX-EVEN-YOU-%E2%9C%85%EF%B8%8F--1155557
scraping: Thread-roblox-cookie-with-limiteds
scraping: Thread-Supreme-Join-discord-gg-kanker
scraping: Thread-%E2%AD%90LEAKED-2023%E2%AD%90%E2%9C%85-ONLYFANS-CHECKER%E2%9C%85-%E2%AD%90FULL-CAPTURE%E2%AD%90%E2%9C%85-100-WORKING%E2%9C%85
scraping: Thread-NEW-IPTV-CONFIG-2024-FULL-CAPTURE-ALL-SERVERS-NO-PROXIES-NEDDED-MAC-PASS
scraping: Thread-HUGE-COLLECTION-OF-UDEMY-COURSES%E2%AD%90-UDEMY-LEAK-%E2%AD%90-Different-Courses-Packed-in-One
scraping: Thread-X138-PREMIUM-DISNEY-HITS
scraping: Thread-Streaming-%E2%AD%903400X-NETFLIX-ACCOUNTS%E2%AD%90FULL-URL-CAPTURE-PREMIUM%E2%AD%90
scraping: Thread-Streaming-%E2%9A%A1-HBO-MAX-%E2%9A%A1-%E2%9A%A1X34-HBOPREMIUM-HITS%E2%AD%90FRESH-HQ%E2%AD%90-